In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import rioxarray
import xarray as xr

In [2]:
tracks = xr.open_dataset(
    './IBTrACS.NA.v04r00.nc'
)

In [3]:
raft_baseline = xr.open_dataset(
    './01_11vars/simulated_real_and_future_TCs/01_11vars_RAFT_baseline.nc'
)
raft_ssp245_near_cold = xr.open_dataset(
    './01_11vars/simulated_real_and_future_TCs/01_11vars_TGW_ssp245_nearFuture_coldModelMean.nc'
)
raft_ssp245_far_cold = xr.open_dataset(
    './01_11vars/simulated_real_and_future_TCs/01_11vars_TGW_ssp245_farFuture_coldModelMean.nc'
)
raft_ssp245_near_hot = xr.open_dataset(
    './01_11vars/simulated_real_and_future_TCs/01_11vars_TGW_ssp245_nearFuture_hotModelMean.nc'
)
raft_ssp245_far_hot = xr.open_dataset(
    './01_11vars/simulated_real_and_future_TCs/01_11vars_TGW_ssp245_farFuture_hotModelMean.nc'
)
raft_ssp585_near_cold = xr.open_dataset(
    './01_11vars/simulated_real_and_future_TCs/01_11vars_TGW_ssp585_nearFuture_coldModelMean.nc'
)
raft_ssp585_far_cold = xr.open_dataset(
    './01_11vars/simulated_real_and_future_TCs/01_11vars_TGW_ssp585_farFuture_coldModelMean.nc'
)
raft_ssp585_near_hot = xr.open_dataset(
    './01_11vars/simulated_real_and_future_TCs/01_11vars_TGW_ssp585_nearFuture_hotModelMean.nc'
)
raft_ssp585_far_hot = xr.open_dataset(
    './01_11vars/simulated_real_and_future_TCs/01_11vars_TGW_ssp585_farFuture_hotModelMean.nc'
)

In [24]:
tracks_df = tracks[['name', 'iso_time', 'dist2land']].to_dataframe().reset_index()
tracks_df['time'] = tracks_df.time.astype('datetime64[s]').astype('datetime64[ns]')
tracks_df['over_land'] = (tracks_df.dist2land == 0)

In [5]:
raft_columns = [
    'stormID', 'datetime_strings', 'vmax_kts',
    'VMPI_t0', 'U200_t0', 'SHRD_t0', 'DELV_6', 'EPTK_t0',
    'LP500_t0', 'PSLV_v3', 'RHLO_t0',
]
raft_df = raft_baseline.to_dataframe().reset_index()[
    raft_columns + ['IBTrACS_vmax']
].rename(columns={
    'IBTrACS_vmax': 'vmax_kts_ibtracs',
}).rename(columns={
    k: f'{k}_baseline' for k in raft_columns[2:]
})
raft_df['datetime_strings'] = pd.to_datetime(raft_df.datetime_strings)
raft_df = raft_df[raft_df.datetime_strings.notna()]

for x, s in [
    (raft_ssp245_near_cold, 'ssp245_near_cold' ),
    (raft_ssp245_far_cold,  'ssp245_far_cold'  ),
    (raft_ssp245_near_hot,  'ssp245_near_hot'  ),
    (raft_ssp245_far_hot,   'ssp245_far_hot'   ),
    (raft_ssp585_near_cold, 'ssp585_near_cold' ),
    (raft_ssp585_far_cold,  'ssp585_far_cold'  ),
    (raft_ssp585_near_hot,  'ssp585_near_hot'  ),
    (raft_ssp585_far_hot,   'ssp585_far_hot'   ),
]:
    df = x.to_dataframe().reset_index()[raft_columns].rename(columns={
        k: f'{k}_{s}' for k in raft_columns[2:]
    })
    df['datetime_strings'] = pd.to_datetime(df.datetime_strings)
    df = df[df.datetime_strings.notna()]
    raft_df = raft_df.merge(
        df,
        how='left',
        on=['stormID', 'datetime_strings'],
    )


In [6]:
raft_df

,stormID,datetime_strings,vmax_kts_baseline,VMPI_t0_baseline,U200_t0_baseline,SHRD_t0_baseline,DELV_6_baseline,EPTK_t0_baseline,LP500_t0_baseline,PSLV_v3_baseline,...,RHLO_t0_ssp585_near_hot,vmax_kts_ssp585_far_hot,VMPI_t0_ssp585_far_hot,U200_t0_ssp585_far_hot,SHRD_t0_ssp585_far_hot,DELV_6_ssp585_far_hot,EPTK_t0_ssp585_far_hot,LP500_t0_ssp585_far_hot,PSLV_v3_ssp585_far_hot,RHLO_t0_ssp585_far_hot
0,1614,1979-06-11 12:00:00,20.000000,131.209167,111.244293,106.563202,1.275106,3549.753174,0.069586,15.633345,...,74.071182,20.000000,133.288132,193.113358,210.648453,0.676450,3674.594971,0.069586,15.633345,74.071182
1,1614,1979-06-11 18:00:00,21.275106,135.788773,64.547928,74.705093,1.894075,3534.800537,0.064632,31.074022,...,72.165237,20.676449,137.867737,146.208939,176.856308,1.660746,3659.642334,0.064632,31.074022,72.165237
2,1614,1979-06-12 00:00:00,23.169180,153.129440,104.456306,102.397736,2.043181,3474.692871,0.092537,15.672303,...,68.081451,22.337196,156.243103,185.948669,203.104538,1.746286,3600.877197,0.092537,15.672303,68.081451
3,1614,1979-06-12 06:00:00,25.212362,152.578140,123.542923,133.145676,1.908759,3482.976807,0.110185,16.392555,...,66.792740,24.083483,155.691818,203.787659,231.827377,1.319693,3609.161133,0.110185,16.392555,66.792740
4,1614,1979-06-12 12:00:00,27.121120,152.578140,142.270966,134.236069,1.581671,3482.976807,0.134008,11.313754,...,64.778915,25.403175,155.691818,220.361084,228.573196,0.460188,3609.161133,0.134008,11.313754,64.778915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17460,2233,2018-11-03 12:00:00,40.285267,66.930473,277.419281,91.035522,-0.990651,2908.239014,0.000000,137.659241,...,84.516556,40.188934,60.894341,373.332611,145.697372,-0.430615,2995.598877,0.000000,137.659241,84.516556
17461,2233,2018-11-03 18:00:00,39.294617,68.977425,256.370667,126.420746,-0.882705,2908.573975,0.000000,141.073364,...,87.792587,39.758320,63.459751,346.679230,177.471878,-0.424790,2999.627930,0.000000,141.073364,87.792587
17462,2233,2018-11-04 00:00:00,38.411911,49.892960,239.093414,131.426331,-1.012827,2892.619873,0.001908,133.319199,...,87.614960,39.333530,43.947140,325.103302,179.880035,-0.734224,2986.276123,0.001908,133.319199,87.614960
17463,2233,2018-11-04 06:00:00,37.399086,52.936192,214.956161,158.444443,-0.785946,2901.711914,0.025934,174.367249,...,86.275253,38.599304,47.814629,296.919708,204.824356,-0.339054,3001.279785,0.025934,174.367249,86.275253


In [25]:
storm_ids = []
storm_names = []
storm_labels = []
storm_years = []
storm_files = []
for storm_id in raft_df.stormID.unique():

    storm = tracks_df[
        (tracks_df.storm==storm_id) &
        tracks_df.time.notna()
    ][['time', 'lat', 'lon', 'name', 'over_land']].reset_index(drop=True).merge(
        raft_df[
            (raft_df.stormID==storm_id)
        ],
        how='left',
        left_on='time',
        right_on='datetime_strings',
    )
    storm_name  = storm.iloc[0]['name'].decode('utf-8').title()
    storm_year  = storm.iloc[0].time.year
    storm_label = f"{storm_name} ({storm_year}) [id:{storm_id}]"
    file_name = f"{storm_name.replace(' ', '_').lower()}_{storm_year}_{storm_id}"
    storm['name'] = storm_name
    storm['year'] = storm_year
    storm['storm_id'] = storm_id
    storm = storm.drop(columns=['datetime_strings', 'stormID'])

    storm_ids.append(storm_id)
    storm_names.append(storm_name)
    storm_labels.append(storm_label)
    storm_years.append(storm_year)
    storm_files.append(file_name)

    schema = pa.Schema.from_pandas(storm, preserve_index=False)
    table = pa.Table.from_pandas(storm, preserve_index=False)
    writer = pa.ipc.new_file(f'../static/data/{file_name}.arrow', schema)
    writer.write(table)
    writer.close()


In [26]:
storm_reference = pd.DataFrame({
    'id': storm_ids,
    'name': storm_names,
    'year': storm_years,
    'label': storm_labels,
    'value': storm_files,
})

In [ ]:
storm_reference.to_json('../src/lib/storms.js', orient='records', indent=2)